# 第五部分 校正 Lexical Semantic（耗时约xx小时）


![](https://ai-studio-static-online.cdn.bcebos.com/cd8aace3dfb34abc97e3cc1d69388ff01d821fb63853487695623d91c2a32307)

## 5.1 Insert N/V/ADJ/ADV...

In [ ]:
# 找到 Word & Phrase - insert n/v/adj/adv...
def identify_insert_word_phrase(data_list_1, data_list_2, data_label_list = None, print_progress = False):
    # 不再处理mispelling和syntactic structure处理过的句子
    misspelling_list = read_candidates_line("common_data/misspelling_list.txt")
    syntactic_structure_list = read_candidates_line("common_data/syntatic_structure_list.txt")
    # replace_0_list = read_candidates_line("replace_0_by_word.txt")
    # replace_1_list = read_candidates_line("replace_1_by_word.txt")
    # 没有特别重要意义的词语
    meanlingless_word_list = ["小布","小爱","小艾","小步","小度","小o","小v","小薇","小冰","app"]
    # [by][Language][How]
    critical_words_list = []
    critibal_connection = ["用","的",""]
    critical_language =["汉语","中文","英语","英文","韩语","韩文","日语","日文","德语","德文","拼音","作文","词组","笔顺","笔画","读音","电话号码"]
    critical_how =["怎么拼","怎拼","怎么说","怎说","怎么读","怎读","怎么写","怎写","是什么",""]      
    for connection in critibal_connection:
        for language in critical_language:
            for how in critical_how:
                critical_words_list.append(connection+language+how)
    critical_words_list = critical_words_list + ["等于多少克","等于多少千克","等于多少斤","等于多少公斤","是什么意思","什么意思","十么意思","怎么拼音"]
    # 确认能够提高的list
    insert_loc_per_org_list = ["LOC","PER","ORG"]
    insert_negate_list = ["不","没","无","莫","非","勿"] # 可与尝试 "是不是" "不是",
    insert_temporal_list = ["t","TIME"] #
    # 不确定是否能够提高的list  存疑，太复杂 
    insert_v_list = ["vd","vn"] # "v" 太多,"vd" 无
    insert_adj_list = ["ad", "an"] #
    insert_n_list = ["n"] #,"n"
    insert_adv_list = ["d"] #
    insert_n_other_list = ["nr","ns","nt","nw","nz"] #
    # 存放修正后的句子对
    insert_1_list = []
    label_1_list = []
    insert_0_list = []
    label_0_list = []
    if print_progress: print("="*15 + " In Progress " + "="*15)
    for i in range (0, len(data_list_1)):
        text_1 = data_list_1[i]
        text_2 = data_list_2[i]
        
        text_1 = text_1.replace(" ","")
        text_2 = text_2.replace(" ","")
        
        # 获取两句话的不同部分
        subset_1, subset_2 = check_subset(text_1,text_2)
        
        text_pair = text_1 + "\t" + text_2

        if text_pair not in (misspelling_list + syntactic_structure_list):# + replace_0_list + replace_1_list): 
            # 若一致则为1
            if text_1 == text_2:
                abc = 1
                insert_1_list.append(data_list_1[i] + "\t" + data_list_2[i])
                if data_label_list!=None:
                    label_1_list.append(str(i) + "\t" + str(data_label_list[i]))
            # 若存在一段不一致的字符串
            elif subset_1!="Not_The_Case" and subset_2!= "Not_The_Case":
                # 找出所有insert的句子对，获取被插入的字符
                insert_words_original = ""
                if subset_1!= "" and subset_2 == "":
                    insert_words_original = subset_1
                elif subset_1=="" and subset_2 !="":
                    insert_words_original = subset_2

                # 当插入的字符大于等于1个字符时
                if len(insert_words_original) >= 1:
                    #分析该插入字符的词性
                    words_insert, nature_insert = get_words_and_words_nature(insert_words_original)
                    
                    # 情况1 判断插入是否是 数字 或者 字母
                    is_insert_contain_only_numeric = True
                    is_insert_contain_only_alpha = True
                    
                    for ni in insert_words_original:
                        # 有些大写字和一容易混淆，排除
                        if not ((ni not in ["什","陌","参","陆","幺"]) and ni.isnumeric()):
                            is_insert_contain_only_numeric = False
                        if not ni.encode('UTF-8').isalpha():
                            is_insert_contain_only_alpha = False
                    
                    # 情况 2 句子对带有需要严格区分内容的
                    # 判断是否包含严格语句
                    is_critical_case_0 = False
                    is_critical_case_1 = False
                    
                    for c_i in range(0, len(critical_words_list)):
                        if critical_words_list[c_i] != "" and critical_words_list[c_i] in text_1 and critical_words_list[c_i] in text_2:

                            # 去掉这些严格词语
                            a = text_1.rstrip(critical_words_list[c_i])
                            b = text_2.rstrip(critical_words_list[c_i])

                            # 剩下的词语 + add words 
                            add_words = ["的","的用","这用","用","这是","是","字"]
                            is_a_or_b_with_add_words = False
                            for a_w in add_words:
                                if (a not in add_words and b not in add_words and a != "" and b!= "") and (a == b or a == b + a_w or a + a_w== b):
                                    is_a_or_b_with_add_words = True
                            # 如果剩下的相等则为1
                            if is_a_or_b_with_add_words:
                                is_critical_case_1 = True
                                break
                            # 如果剩下的不等则为0
                            else:
                                is_critical_case_0 = True
                                break
            
                    # 情况 3 同一个内容在一个句子中给重复出现多次
                    # 判断 text_1 和 text_2 是否有加减乘除
                    is_contain_jzcc = False
                    for jzcc_word in ["加","减","乘","除"]:
                        if jzcc_word in text_1 or jzcc_word in text_2:
                            is_contain_jzcc = True
                    # 判断 text_1 和 text_2 是否有数字
                    is_contain_number = False
                    for number_word in ["一","二","三","四","五","六","七","八","九","十","1","2","3","4","5","6","7","8","9","0"]:
                        if number_word in text_1 or number_word in text_2:
                            is_contain_number = True
                    # 判断 text_1 和 text_2 是否都是汉字
                    is_text_1_all_chinese = True
                    is_text_2_all_chinese = True
                    for ch in text_1:
                        if not ('\u4e00' <= ch <= '\u9fa5'):
                            is_text_1_all_chinese = False
                    for ch in text_2:
                        if not ('\u4e00' <= ch <= '\u9fa5'):
                            is_text_2_all_chinese = False
                    # 判断是否重复内容
                    is_repeat_content = False
                    elements_in_text_1 = list(set(text_1))
                    elements_in_text_2 = list(set(text_2))
                    new_text_1 = ""
                    new_text_2 = ""
                    for element in elements_in_text_1:
                        new_text_1 = new_text_1 + element
                    for element in elements_in_text_2:
                        new_text_2 = new_text_2 + element
                    if new_text_1 == new_text_2 and new_text_1 in text_2 and new_text_2 in text_1 and not (is_contain_jzcc and is_contain_number) :
                        is_repeat_content = True

                    # 情况4 鲁棒性增强句子对的判断
                    is_text_1_contain_number = False
                    is_text_2_contain_number = False
                    for number_word in ["三","四","五","六","七","八","九","十","1","2","3","4","5","6","7","8","9","0"]:
                        if number_word in text_1:
                            is_text_1_contain_number = True
                        if number_word in text_2:
                            is_text_2_contain_number = True
                    
                    # 如果插入的字词已经在两个句子中 (Verified train)
                    if insert_words_original in meanlingless_word_list:
                        abc = 1
                        insert_1_list.append(data_list_1[i] + "\t" + data_list_2[i])
                        if data_label_list!=None:
                            label_1_list.append(str(i) + "\t" + str(data_label_list[i]))
                    # 仅插入这些字符
                    elif len(insert_words_original) == 1 and ((insert_words_original in ["三","四","五","六","七","八","九","十","1","2","3","4","5","6","7","8","9","0"]) and (not (is_text_1_contain_number and is_text_2_contain_number))) or is_insert_contain_only_alpha:
                        abc = 1
                        insert_0_list.append(data_list_1[i] + "\t" + data_list_2[i])
                        if data_label_list!=None:
                            label_0_list.append(str(i) + "\t" + str(data_label_list[i]))
                    # Insert Negate 如果包含 否定词，且 否定词 不在最后一个字 (verfified train/test)
                    elif insert_words_original in insert_negate_list:
                        abc = 1
                        # 如果否定词在最后可以表示问句
                        if  text_1[-1] == "不" or text_2[-1] == "不" or "不我" in text_1 or "不我" in text_2 or "不你" in text_1 or "不你" in text_2 or "不帮我" in text_1 or "不帮我" in text_2 or "不小布" in text_1 or "不小布" in text_2:
                            insert_1_list.append(data_list_1[i] + "\t" + data_list_2[i])
                            if data_label_list!=None:
                                label_1_list.append(str(i) + "\t" + str(data_label_list[i]))
                        else:
                            insert_0_list.append(data_list_1[i] + "\t" + data_list_2[i])
                            if data_label_list!=None:
                                label_0_list.append(str(i) + "\t" + str(data_label_list[i]))
                    # Insert temporal (verfified train)
                    elif len(words_insert) == 1 and nature_insert[0] in insert_temporal_list:
                        is_current_time = insert_words_original in ["现在","今天","今日","早上","目前","当前","此时","平时","最近","当前","今年","每日","每天","近期","此刻","唐","宋","元","明","清"]
                        # 是否是需要严格区分的情况 如用英语怎么说
                        if is_critical_case_0 and is_current_time:
                            insert_0_list.append(data_list_1[i] + "\t" + data_list_2[i])
                            if data_label_list!=None:
                                label_0_list.append(str(i) + "\t" + str(data_label_list[i])) 
                        # 一种是插入了默认的时间点，另一种是同一个时间点的重复（当然要排除数学计算上的误判，如二点五加二点五）
                        elif is_current_time or ((not insert_words_original[0].isnumeric()) and (insert_words_original in text_1 and insert_words_original in text_2)) :
                            insert_1_list.append(data_list_1[i] + "\t" + data_list_2[i])
                            if data_label_list!=None:
                                label_1_list.append(str(i) + "\t" + str(data_label_list[i]))
                        # 其余时间信息则为 0
                        else:
                            insert_0_list.append(data_list_1[i] + "\t" + data_list_2[i])
                            if data_label_list!=None:
                                label_0_list.append(str(i) + "\t" + str(data_label_list[i]))   
                    # Insert LOC/PER/ORG (verfified train)(混乱)
                    elif len(words_insert) == 1 and nature_insert[0] in insert_loc_per_org_list:
                        abc = 1 
                        if (insert_words_original in text_1 and insert_words_original in text_2) or (insert_words_original in ["省","市","镇","乡","县","村","岛","区"]):
                            insert_1_list.append(data_list_1[i] + "\t" + data_list_2[i])
                            if data_label_list!=None:
                                label_1_list.append(str(i) + "\t" + str(data_label_list[i])) 
                        else:
                            insert_0_list.append(data_list_1[i] + "\t" + data_list_2[i])
                            if data_label_list!=None:
                                label_0_list.append(str(i) + "\t" + str(data_label_list[i])) 
                    # # Insert v  (混乱)
                    # elif len(insert_words_original) >= 2 and len(words_insert) == 1 and nature_insert[0] in insert_v_list:
                    #     abc = 1
                    #     # if (insert_words_original in text_1 and insert_words_original in text_2) :
                    #     #     insert_1_list.append(data_list_1[i] + "\t" + data_list_2[i])
                    #     #     if data_label_list!=None:
                    #     #         label_1_list.append(str(i) + "\t" + str(data_label_list[i]))
                    #     # else: 
                    #     #     insert_0_list.append(data_list_1[i] + "\t" + data_list_2[i])
                    #     #     if data_label_list!=None:
                    #     #         label_0_list.append(str(i) + "\t" + str(data_label_list[i]))
                    # # Insert adj  (混乱)
                    # elif len(words_insert) == 1 and nature_insert[0] in insert_adj_list:
                    #     abc = 1
                    #     # insert_0_list.append(data_list_1[i] + "\t" + data_list_2[i])
                    #     # if data_label_list!=None:
                    #     #     label_0_list.append(str(i) + "\t" + str(data_label_list[i]))
                    # # Insert adv  (混乱)
                    # elif len(insert_words_original) >= 1 and len(words_insert) == 1 and nature_insert[0] in insert_adv_list:
                    #     abc = 1 
                    #     # if (insert_words_original in ["到底","很","确实","其实","非常","就","都","究竟","马上","一起","的确","也","已经","居然","赶快","太","一共", "又", "再", "难道", "当然", "实在", "真就", "大概", "赶紧", "简直","立刻", "难怪", "老是", "从来", "才", "好好", "大约", "好像", "先", "总是", "永远"]) or (insert_words_original in text_1 and insert_words_original in text_2) :
                    #     #     insert_1_list.append(data_list_1[i] + "\t" + data_list_2[i])
                    #     #     if data_label_list!=None:
                    #     #         label_1_list.append(str(i) + "\t" + str(data_label_list[i]))
                    #     # elif (insert_words_original in ["一直"]) :
                    #     #     insert_1_list.append(data_list_1[i] + "\t" + data_list_2[i])
                    #     #     if data_label_list!=None:
                    #     #         label_1_list.append(str(i) + "\t" + str(data_label_list[i]))
                    #     # else: 
                    #     #     insert_0_list.append(data_list_1[i] + "\t" + data_list_2[i])
                    #     #     if data_label_list!=None:
                    #     #         label_0_list.append(str(i) + "\t" + str(data_label_list[i]))  
                    # # Insert n (混乱)
                    # elif len(insert_words_original) >= 2 and len(words_insert) == 1 and nature_insert[0] in insert_n_list:
                    #     abc = 1  #手机
                    #     # if (insert_words_original in ["天下","意思","","","","",""]) or (insert_words_original in text_1 and insert_words_original in text_2) :
                    #     #     insert_1_list.append(data_list_1[i] + "\t" + data_list_2[i])
                    #     #     if data_label_list!=None:
                    #     #         label_1_list.append(str(i) + "\t" + str(data_label_list[i]))
                    #     # else: 
                    #     #     insert_0_list.append(data_list_1[i] + "\t" + data_list_2[i])
                    #     #     if data_label_list!=None:
                    #     #         label_0_list.append(str(i) + "\t" + str(data_label_list[i])) 
                
    
        if print_progress and i%10000 == 0:
            print("%.2f of text pairs for insert subset identification have processed " %((i/len(data_list_1))*100))    
    if print_progress: print("="*15 + "     Done    " + "="*15)
    print("插入 指定 natures 的句子对共 " + str(len(insert_1_list) + len(insert_0_list)) + " 组\n")
    return insert_1_list, label_1_list, insert_0_list, label_0_list

In [ ]:
# 寻找Insert的句子对
# 准备句子对和标签
test_text_pair_1_list, test_text_pair_2_list = read_candidates_test("work/test_B_processed.txt")
view_label_list = read_labels("predict_results/ms_ss_ccf_qianyan_qm_result_B.csv")

# 寻找包含指定字或字符的句子对
insert_1_list, label_1_list, insert_0_list, label_0_list = identify_insert_word_phrase(test_text_pair_1_list,test_text_pair_2_list, view_label_list, True)
write_file("common_data/insert_1_by_word.txt",insert_1_list)
write_file("common_data/insert_0_by_word.txt",insert_0_list)

# 打印结果
print_dataset(insert_1_list, label_1_list,300)
print_dataset(insert_0_list, label_0_list,300)


The number of lines in work/test_B_processed.txt is 100000

The number of predict result lines in predict_results/ms_ss_ccf_qianyan_qm_result_B.csv is 100000
=============== In Progress ===============
0.00 of text pairs for insert subset identification have processed 
10.00 of text pairs for insert subset identification have processed 
20.00 of text pairs for insert subset identification have processed 
30.00 of text pairs for insert subset identification have processed 
40.00 of text pairs for insert subset identification have processed 
50.00 of text pairs for insert subset identification have processed 
60.00 of text pairs for insert subset identification have processed 
70.00 of text pairs for insert subset identification have processed 
80.00 of text pairs for insert subset identification have processed 
90.00 of text pairs for insert subset identification have processed 
===============     Done    ===============
插入 指定 natures 的句子对共 873 组

===============     统计    ===========

## 5.2 Replace N/V/ADJ/ADV...

In [ ]:
# 找到 Word & Phrase - replace n/v/adj/adv...
def identify_replace_word_phrase(data_list_1, data_list_2, data_label_list = None, print_progress = False):
    # 不再处理mispelling和syntactic structure处理过的句子
    misspelling_list = read_candidates_line("common_data/misspelling_list.txt")
    syntactic_structure_list = read_candidates_line("common_data/syntatic_structure_list.txt")
    insert_0_list = read_candidates_line("common_data/insert_0_by_word.txt")
    insert_1_list = read_candidates_line("common_data/insert_1_by_word.txt")
    
    # 载入 同义词 列表
    synonym_list = read_candidates_line("common_data/synonym_list.txt")
    print("="*15 + "synonym_list" + "="*15)
    print(len(synonym_list))
    # [by][Language][How]
    critical_words_list = []
    critibal_connection = ["用","的",""]
    critical_language =["汉语","中文","英语","英文","韩语","韩文","日语","日文","德语","德文","拼音","作文","词组","笔顺","笔画","读音","电话号码","英文翻译","英语翻译","英文单词","英语单词",""]
    critical_how =["怎么拼","怎拼","怎么说","怎说","怎么读","怎读","怎么写","怎写","是什么",""]      
    for connection in critibal_connection:
        for language in critical_language:
            for how in critical_how:
                if connection+language+how not in critibal_connection:
                    critical_words_list.append(connection+language+how)
    critical_words_list = critical_words_list + ["等于多少克","等于多少千克","等于多少斤","等于多少公斤","是什么意思","什么意思","十么意思","怎么拼音"]
    # 各类关注的词性
    # Named Entity
    replace_per_nr_list = ["PER","nr"]
    replace_loc_ns_list = ["LOC","ns"]
    replace_org_nt_list = ["ORG","nt"]
    replace_nw_nz_list = ["nw","nz"]
    # Word/Pharse/Name Entity
    replace_n_list = ["n"] # "n" 
    # Word/Pharse/Synonym
    replace_v_list = ["vd","vn"] #"v","vd" 无,"vn","v",
    # adj/adv/time
    replace_adj_list = ["a", "ad", "an"] # 
    replace_adv_list = ["d"] #
    replace_temporal_list = ["t","TIME"] #

    replace_1_list = []
    label_1_list = []
    replace_0_list = []
    label_0_list = []

    if print_progress: print("="*15 + " In Progress " + "="*15)
    
    for i in range (0, len(data_list_1)):
        text_1 = data_list_1[i]
        text_2 = data_list_2[i]
                
        # 若为 已经处理过的情况，则跳过
        if data_list_1[i] + "\t" + data_list_2[i] in (misspelling_list + misspelling_list + insert_0_list + insert_1_list):
            abc = 1
        # 完全一致的为 跳过
        elif text_1 == text_2:
            abc = 1
            replace_1_list.append(data_list_1[i] + "\t" + data_list_2[i])
            if data_label_list!=None:
                label_1_list.append(str(i) + "\t" + str(data_label_list[i])) 
        # 进入replace的处理
        else:  

            # 句子对带有需要严格区分内容的
            # 判断是否包含严格语句
            is_critical_case_0 = False
            is_critical_case_1 = False
                    
            for c_i in range(0, len(critical_words_list)):
                if critical_words_list[c_i] != "" and critical_words_list[c_i] in text_1 and critical_words_list[c_i] in text_2:

                    # 去掉这些严格词语
                    a = text_1.rstrip(critical_words_list[c_i])
                    b = text_2.rstrip(critical_words_list[c_i])
                    # 剩下的词语 + add words 
                    add_words = ["的","的用","这用","用","这是","是","字"]
                    is_a_or_b_with_add_words = False
                    for a_w in add_words:
                        if (a not in add_words and b not in add_words and a != "" and b!= "") and (a == b or a == b + a_w or a + a_w== b):
                            is_a_or_b_with_add_words = True
                    # 如果剩下的相等则为1
                    if is_a_or_b_with_add_words:
                        is_critical_case_1 = True
                        break
                    # 如果剩下的不等则为0
                    else:
                        is_critical_case_0 = True
                        break

            # (1) 获取两句话中被替换的部分 by word 
            subset_1, subset_2 = check_subset(text_1,text_2)
            # 判断插入是否是 字母
            is_replace_contain_only_alpha_1 = True
            is_replace_contain_only_alpha_2 = True
            is_replace_contain_only_numeric_1 = True 
            is_replace_contain_only_numeric_2 = True 

            for ni in subset_1:
                if not ((ni not in ["什","陌","参","陆","幺"]) and ni.isnumeric()):
                    is_replace_contain_only_numeric_1 = False
                if not ni.encode('UTF-8').isalpha():
                    is_replace_contain_only_alpha_1 = False
            for ni in subset_2:
                if not ((ni not in ["什","陌","参","陆","幺"]) and ni.isnumeric()):
                    is_replace_contain_only_numeric_2 = False
                if not ni.encode('UTF-8').isalpha():
                    is_replace_contain_only_alpha_2 = False

            s_w_1, s_n_1 = get_words_and_words_nature(subset_1)
            s_w_2, s_n_2 = get_words_and_words_nature(subset_2)

            # (2) 获取两句话中被替换的部分 by nature
            words_1, nature_1 = get_words_and_words_nature(text_1)
            words_2, nature_2 = get_words_and_words_nature(text_2)

            words_natures_1 = merge_word_nature (words_1, nature_1)
            words_natures_2 = merge_word_nature (words_2, nature_2)

            subset_word_nature_1, subset_word_nature_2 = check_subset(words_natures_1,words_natures_2)
            # print( subset_word_nature_1, subset_word_nature_2 )
            # print( is_critical_case_0 )
            # by word
            # 如果替换的是字母
            if subset_1!="Not_The_Case" and subset_2!= "Not_The_Case" and subset_1 != "" and subset_2 != "" and is_replace_contain_only_alpha_1 and is_replace_contain_only_alpha_2:
                abc = 1
                replace_0_list.append(data_list_1[i] + "\t" + data_list_2[i])
                if data_label_list!=None:
                    label_0_list.append(str(i) + "\t" + str(data_label_list[i])) 
            # 如果替换的是数字
            elif subset_1!="Not_The_Case" and subset_2!= "Not_The_Case" and subset_1 != "" and subset_2 != "" and is_replace_contain_only_numeric_1 and is_replace_contain_only_numeric_2:
                abc = 1
                replace_0_list.append(data_list_1[i] + "\t" + data_list_2[i])
                if data_label_list!=None:
                    label_0_list.append(str(i) + "\t" + str(data_label_list[i])) 
            # 如果替换（by word）对出现在近义词表中
            elif (subset_1!="Not_The_Case") and (subset_2!= "Not_The_Case") and ((subset_1 + "\\t" + subset_2) in synonym_list or (subset_2 + "\\t" + subset_1) in synonym_list) and not is_critical_case_0:
                abc = 1
                replace_1_list.append(data_list_1[i] + "\t" + data_list_2[i])
                if data_label_list!=None:
                    label_1_list.append(str(i) + "\t" + str(data_label_list[i])) 
            # by nature
            elif len(subset_word_nature_1) == 1 and len(subset_word_nature_2) == 1 and subset_word_nature_1!="Not_The_Case" and subset_word_nature_2!= "Not_The_Case" and subset_word_nature_1 != "" and subset_word_nature_2 != "" and not is_critical_case_0:    
                t_1 = subset_word_nature_1[0][0] + "\\t" + subset_word_nature_2[0][0] 
                t_2 = subset_word_nature_2[0][0] + "\\t" + subset_word_nature_1[0][0]       
                # 如果 是同义词表中的替换词则为1 
                if t_1 in synonym_list or t_2 in synonym_list:
                    abc =1
                    replace_1_list.append(data_list_1[i] + "\t" + data_list_2[i])
                    if data_label_list!=None:
                        label_1_list.append(str(i) + "\t" + str(data_label_list[i])) 
                # replace per
                elif subset_word_nature_1[0][1] in replace_per_nr_list and subset_word_nature_2[0][1] in replace_per_nr_list:
                    abc = 1
                    replace_0_list.append(data_list_1[i] + "\t" + data_list_2[i])
                    if data_label_list!=None:
                        label_0_list.append(str(i) + "\t" + str(data_label_list[i]))  
                # Replace LOC nr
                elif subset_word_nature_1[0][1] in replace_loc_ns_list and subset_word_nature_2[0][1] in replace_loc_ns_list:
                    is_loc_full = False
                    for loc_word in ["国","省","市","镇","乡","县","村","岛","区","景区","林区","地区"]:
                        if  subset_word_nature_1[0][0] == subset_word_nature_2[0][0] + loc_word:
                            is_loc_full = True
                        elif  subset_word_nature_1[0][0]+ loc_word == subset_word_nature_2[0][0] :
                            is_loc_full = True   
                    if is_loc_full:
                        replace_1_list.append(data_list_1[i] + "\t" + data_list_2[i])
                        if data_label_list!=None:
                            label_1_list.append(str(i) + "\t" + str(data_label_list[i]))  
                    else:
                        replace_0_list.append(data_list_1[i] + "\t" + data_list_2[i])
                        if data_label_list!=None:
                            label_0_list.append(str(i) + "\t" + str(data_label_list[i]))  
                # Replace ORG nt
                elif subset_word_nature_1[0][1] in replace_org_nt_list and subset_word_nature_2[0][1] in replace_org_nt_list:
                    is_org_full = False
                    s_1,s_2 = check_subset(subset_word_nature_1[0][0],subset_word_nature_2[0][0])
                    org_words = ["公司","集团","局","部门","处","省","市","镇","乡","县","岛","区"]
                    if s_1 in org_words or s_2 in org_words:
                        is_org_full = True
                    if (s_1+"\\t"+s_2) in synonym_list or (s_2 + "\\t" + s_1) in synonym_list:
                        is_org_full = True
                    if is_org_full:
                        replace_1_list.append(data_list_1[i] + "\t" + data_list_2[i])
                        if data_label_list!=None:
                            label_1_list.append(str(i) + "\t" + str(data_label_list[i]))  
                    else:
                        replace_0_list.append(data_list_1[i] + "\t" + data_list_2[i])
                        if data_label_list!=None:
                            label_0_list.append(str(i) + "\t" + str(data_label_list[i]))  
            #     # Replace NW/NZ (混乱)
            #     # elif subset_word_nature_1[0][1] in replace_nw_nz_list and subset_word_nature_2[0][1] in replace_nw_nz_list:
            #     #     replace_0_list.append(data_list_1[i] + "\t" + data_list_2[i])
            #     #     if data_label_list!=None:
            #     #         label_0_list.append(str(i) + "\t" + str(data_label_list[i]))
            #     # Replace N  (混乱)
            #     # elif subset_word_nature_1[0][1] in replace_n_list and subset_word_nature_2[0][1] in replace_n_list:
            #     #     replace_0_list.append(data_list_1[i] + "\t" + data_list_2[i])
            #     #     if data_label_list!=None:
            #     #         label_0_list.append(str(i) + "\t" + str(data_label_list[i]))
            #     # Replace V（混乱）
            #     # elif subset_word_nature_1[0][1] in replace_v_list and subset_word_nature_2[0][1] in replace_v_list:
            #     #     replace_0_list.append(data_list_1[i] + "\t" + data_list_2[i])
            #     #     if data_label_list!=None:
            #     #         label_0_list.append(str(i) + "\t" + str(data_label_list[i]))
            #     # Replace adj（混乱）
            #     # elif subset_word_nature_1[0][1] in replace_adj_list and subset_word_nature_2[0][1] in replace_adj_list:
            #     #     replace_0_list.append(data_list_1[i] + "\t" + data_list_2[i])
            #     #     if data_label_list!=None:
            #     #         label_0_list.append(str(i) + "\t" + str(data_label_list[i]))
                # Replace adv
                elif subset_word_nature_1[0][1] in replace_adv_list and subset_word_nature_2[0][1] in replace_adv_list:
                    abc = 1
                    replace_0_list.append(data_list_1[i] + "\t" + data_list_2[i])
                    if data_label_list!=None:
                        label_0_list.append(str(i) + "\t" + str(data_label_list[i]))
                # Replace temporal
                elif subset_word_nature_1[0][1] in replace_temporal_list and subset_word_nature_2[0][1] in replace_temporal_list:
                    is_temp_full = False
                    s_1,s_2 = check_subset(subset_word_nature_1[0][0],subset_word_nature_2[0][0])
                    temp_words = ["月","日","份","个","号","钟"]
                    if s_1 in temp_words or s_2 in temp_words:
                        is_temp_full = True
                    if (s_1+"\\t"+s_2) in synonym_list or (s_2 + "\\t" + s_1) in synonym_list:
                        is_temp_full = True
                    if is_temp_full and not is_critical_case_0:
                        replace_1_list.append(data_list_1[i] + "\t" + data_list_2[i])
                        if data_label_list!=None:
                            label_1_list.append(str(i) + "\t" + str(data_label_list[i]))  
                    else:
                        replace_0_list.append(data_list_1[i] + "\t" + data_list_2[i])
                        if data_label_list!=None:
                            label_0_list.append(str(i) + "\t" + str(data_label_list[i]))                
    
        if print_progress and i%10000 == 0:
            print("%.2f of text pairs for replace subset identification have processed " %((i/len(data_list_1))*100))    
    if print_progress: print("="*15 + "     Done    " + "="*15)
    print("替换 指定 natures 的句子对共 " + str(len(replace_1_list) + len(replace_0_list)) + " 组\n")
    return replace_1_list, label_1_list, replace_0_list, label_0_list

In [ ]:
# 寻找Replace的句子对
# 准备句子对和标签
test_text_pair_1_list, test_text_pair_2_list = read_candidates_test("work/test_B_processed.txt")
view_label_list = read_labels("predict_results/ms_ss_ccf_qianyan_qm_result_B.csv")

# 寻找包含指定字或字符的句子对
replace_1_list, label_1_list, replace_0_list, label_0_list = identify_replace_word_phrase(test_text_pair_1_list,test_text_pair_2_list, view_label_list, True)
write_file("common_data/replace_1_by_word.txt",replace_1_list)
write_file("common_data/replace_0_by_word.txt",replace_0_list)

# 打印结果
print_dataset(replace_1_list, label_1_list,50)
print_dataset(replace_0_list, label_0_list,len(replace_0_list))

句子对: 十八字短句	三字短句	86807	0
句子对: 正月十五的习俗急啊	腊月十五的习俗急啊	86812	0
句子对: 张羽茉名字打分	张梦琪名字打分	86813	0
句子对: 苹果六代替home键的小圆点哪里设置	苹果十一代替home键的小圆点哪里设置	86825	0
句子对: 什么是肿瘤四项	什么是肿瘤六项	86852	0
句子对: 战地五内存要求	战地一内存要求	86862	0
句子对: 历史上姓翟的名人有哪些	历史上姓苏的名人有哪些	86889	0
句子对: 八十五除一百	八十五除二	86891	0
句子对: 一百乘二十	五百乘二十	86902	0
句子对: 赣马高级中学校长	潮州高级中学校长	86921	0
句子对: 月经干净后第一天会排卵吗	月经干净后第九天会排卵吗	86942	0
句子对: 阴历三月初四是什么命	阴历四月初四是什么命	86943	0
句子对: 云南警官学院怎么样	云南中医学院怎么样	86950	0
句子对: 中行客服工资	中信客服工资	86967	0
句子对: 四川师范大学校训	广西师范大学校训	86981	0
句子对: 十一年年三十是多少号	十二年年三十是多少号	86994	0
句子对: 大写七字怎么写笔顺	大写一字怎么写笔顺	87002	0
句子对: 山东方言	内蒙古方言	87018	0
句子对: 九百五十一除五的竖式怎么列	零除五的竖式怎么列	87029	0
句子对: 二十乘八	二十乘六	87045	0
句子对: 齐齐哈尔到昂昂溪多少公里	齐齐哈尔到郑州多少公里	87059	0
句子对: 和田到张掖的飞机票	和田到天津的飞机票	87074	0
句子对: 苏州到沈阳火车票价格	北京到沈阳火车票价格	87107	0
句子对: 山西师范大学是双一流吗	陕西师大是双一流吗	87126	0
句子对: 上海交大和北大哪个好	上海交大和人大哪个好	87135	0
句子对: 形容青春的一个字	形容青春的四个字	87157	0
句子对: 三十八年出生男演员	三十九年出生男演员	87164	0
句子对: 三月十一日是什么星座	三月九日是什么星座	87168	0
句子对: 第九的英语怎么说	第二的英语怎么说	87175	0
句子对: 八千开根号等多少	一万三千开根号等多少	87198	0
句子对: 太原到湖州火车时刻表	太原到南昌火车时刻表	872

## 5.3 校正 Lexical Semantic 预测值

In [ ]:
# test a text pairs and predict labels
test_a_data_line_list = read_candidates_line("work/test_B_processed.txt")
print("test_a_data_line_list: " + str(len(test_a_data_line_list)))

predict_label_list = read_labels("predict_results/ms_ss_ccf_qianyan_qm_result_B.csv")
print("predict_label_list: " + str(len(predict_label_list)))

# symmetry 读取句子对
insert_0_list = read_candidates_line("common_data/insert_0_by_word.txt")
insert_1_list = read_candidates_line("common_data/insert_1_by_word.txt")
replace_0_list = read_candidates_line("common_data/replace_0_by_word.txt")
replace_1_list = read_candidates_line("common_data/replace_1_by_word.txt")

to_be_1_list = insert_1_list + replace_1_list
to_be_0_list = insert_0_list + replace_0_list

lexical_semantic_list = to_be_0_list + to_be_1_list
print("identify " + str(len(lexical_semantic_list)) + " pairs of lexical semantic")
write_file("common_data/lexical_semantic_list.txt", lexical_semantic_list)

# 赋值
adjusted_predict_0_list_index_label_list = []
for i in range (0, len(test_a_data_line_list)):
    if test_a_data_line_list[i] in to_be_0_list:
        adjusted_predict_0_list_index_label_list.append(str(i)+"\t0")

adjusted_predict_1_list_index_label_list = []
for i in range (0, len(test_a_data_line_list)):
    if test_a_data_line_list[i] in to_be_1_list:
        adjusted_predict_1_list_index_label_list.append(str(i)+"\t1")

adjusted_predict_list_index_label_list = adjusted_predict_0_list_index_label_list + adjusted_predict_1_list_index_label_list

# 使用 方法4-1 来 校正 预测值 
final_predict_lables = adjust_predict_label(test_a_data_line_list, predict_label_list, adjusted_predict_list_index_label_list, True)

# 写入文件
write_file("predict_results/ms_ss_ls_ccf_qianyan_qm_result_B.csv",final_predict_lables)

predict_label:1
adjusted_label:0
怎样在手机上下载视频	怎样在苹果手机上下载视频
index:46686
predict_label:1
adjusted_label:0
qq群宣传语怎么写	群宣传语怎么写
index:47106
predict_label:1
adjusted_label:0
小米怎么进去recovery模式	小米怎么进去recover模式
index:47336
predict_label:1
adjusted_label:0
阿里巴巴的股价	阿里的股价
index:47816
predict_label:1
adjusted_label:0
开地产公司要多少钱	开房地产公司要多少钱
index:48195
predict_label:1
adjusted_label:0
伯利特股票发行价	伯特利股票发行价
index:48739
predict_label:1
adjusted_label:0
五行属火的吉利字	五行属火的字
index:50388
predict_label:1
adjusted_label:0
石家庄到大寨多少公里	石家庄到山西大寨多少公里
index:50730
predict_label:1
adjusted_label:0
邮政国际邮费价格表	中国邮政国际邮费价格表
index:50929
predict_label:1
adjusted_label:0
彭州二级医院有哪些	彭州二甲医院有哪些
index:51174
predict_label:1
adjusted_label:0
复旦成考录取分数线	上海复旦成考录取分数线
index:51260
predict_label:1
adjusted_label:0
大便一直不正常	大便不不正常
index:51533
predict_label:1
adjusted_label:0
喉咙里好像有异物感	喉咙里一直有异物感
index:51612
predict_label:1
adjusted_label:0
腰围七十六cm等多少尺	腰围七十六等多少尺
index:52119
predict_label:1
adjusted_label:0
广东移动短号怎么取消	移动短号怎么取消
index:52268
predict_label:1


## 5.4 对比校正结果

In [ ]:
#读取test和两个预测值
test_text_list_1, test_text_list_2 = read_candidates_test("work/test_B_1118.tsv")
test_predict_label_list_1 = read_labels("predict_results/ms_ss_ccf_qianyan_qm_result_B.csv")
test_predict_label_list_2 = read_labels("predict_results/ms_ss_ls_ccf_qianyan_qm_result_B.csv")
print(len(test_text_list_1))
print(len(test_text_list_2))
print(len(test_predict_label_list_1))
print(len(test_predict_label_list_2))

# 对比两组预测值，并输出不同的预测结果
def compre_predict_results(text_list1, text_list2, candidates_1, candidates_2):
    list1 = []
    list2 = []
    label1 = []
    label2 = []
    j = 0
    for i in range (0,len(candidates_1)): 
        if candidates_1[i] != candidates_2[i]:
            print("No. " + str(i) + " label 1 = " + str(candidates_1[i]) + " lable 2 = " + str(candidates_2[i]) + " text pair: " + text_list1[i] + " " + text_list2[i])
            list1.append(text_list1[i])
            list2.append(text_list2[i])
            label1.append(candidates_1[i])
            label2.append(candidates_2[i])
            j = j + 1
    print("Total number of differences is " + str(j))
    print("Two prediction's similarity is %.2f"%((len(candidates_2) - j)/len(candidates_2)*100))
    return list1, list2, label1, label2

text_list1, text_list2, correct_text_list1, predict_text_list1= compre_predict_results(test_text_list_1, test_text_list_2, test_predict_label_list_1, test_predict_label_list_2)
print("不同的预测值共有： " + str(len(predict_text_list1)) + " 组")


The number of lines in work/test_B_1118.tsv is 100000

The number of predict result lines in predict_results/ms_ss_ccf_qianyan_qm_result_B.csv is 100000

The number of predict result lines in predict_results/ms_ss_ls_ccf_qianyan_qm_result_B.csv is 100000
100000
100000
100000
100000
No. 48 label 1 = 1 lable 2 = 0 text pair: 我是不是真的有点烦人 我是不是真的很烦人
No. 650 label 1 = 1 lable 2 = 0 text pair: 雷克萨斯ls怎么样 雷克萨斯lc怎么样
No. 681 label 1 = 0 lable 2 = 1 text pair: 美国总统是谁 美国总统现在是谁
No. 801 label 1 = 1 lable 2 = 0 text pair: 苹果手机7p屏幕上的悬浮球怎么设置 苹果手机7屏幕上的悬浮球怎么设置
No. 853 label 1 = 1 lable 2 = 0 text pair: 华西医学院院长 华西医院院长
No. 1100 label 1 = 1 lable 2 = 0 text pair: 我为什么不信基督教 我为什么不再信基督教
No. 1530 label 1 = 0 lable 2 = 1 text pair: 退出开车模式 退出车载模式
No. 1885 label 1 = 1 lable 2 = 0 text pair: 北京地坛医院新址在哪 天坛医院新址在哪
No. 2068 label 1 = 1 lable 2 = 0 text pair: 热巴为什么被新浪限流 迪丽热巴为什么被新浪限流
No. 2325 label 1 = 0 lable 2 = 1 text pair: 我可以睡觉吗 我现在可以睡觉吗
No. 2481 label 1 = 1 lable 2 = 0 text pair: 郑州轻工业学院好不好 郑州轻工业大学好不好
No. 2889 label